In [1]:
import numpy as np
import pandas as pd
import string 
import re

In [2]:
with open('businessCard.txt',mode='r',encoding='utf8',errors='ignore') as f:
    text = f.read()

In [3]:
data = list(map(lambda x:x.split('\t'),text.split('\n')))

In [4]:
data

[['id', 'text', 'tag', '', ''],
 ['000.jpeg', ' ', 'O', '', ''],
 ['000.jpeg', '.', 'O', '', ''],
 ['000.jpeg', '040-4852', 'B-PHONE', '', ''],
 ['000.jpeg', '"8881,"', 'I-PHONE', '', ''],
 ['000.jpeg', '90309', 'B-PHONE', '', ''],
 ['000.jpeg', '52549', 'I-PHONE', '', ''],
 ['000.jpeg', 'Fi', 'O', '', ''],
 ['000.jpeg', '/laurelsoverseaseducation', 'O', '', ''],
 ['000.jpeg', 'LAURELS', 'B-ORG', '', ''],
 ['000.jpeg', 'OVERSEAS', 'I-ORG', '', ''],
 ['000.jpeg', 'EDUCATIONAL', 'I-ORG', '', ''],
 ['000.jpeg', 'CONSULTANCY', 'I-ORG', '', ''],
 ['000.jpeg', 'PVT.', 'I-ORG', '', ''],
 ['000.jpeg', 'LTD.', 'I-ORG', '', ''],
 ['000.jpeg', 'Sea', 'O', '', ''],
 ['000.jpeg', '|', 'O', '', ''],
 ['000.jpeg', 'U.K', 'O', '', ''],
 ['000.jpeg', 'AUSTRALIA', 'O', '', ''],
 ['000.jpeg', 'CANADA', 'O', '', ''],
 ['000.jpeg', 'IRELAND', 'O', '', ''],
 ['000.jpeg', ' ', 'O', '', ''],
 ['000.jpeg', ' ', 'O', '', ''],
 ['000.jpeg', ' ', 'O', '', ''],
 ['000.jpeg', ' ', 'O', '', ''],
 ['000.jpeg', ' ', '

In [5]:
df = pd.DataFrame(data[1:],columns=data[0])

In [6]:
df.head(10)

,id,text,tag,,
0,000.jpeg,,O,,
1,000.jpeg,.,O,,
2,000.jpeg,040-4852,B-PHONE,,
3,000.jpeg,"""8881,""",I-PHONE,,
4,000.jpeg,90309,B-PHONE,,
5,000.jpeg,52549,I-PHONE,,
6,000.jpeg,Fi,O,,
7,000.jpeg,/laurelsoverseaseducation,O,,
8,000.jpeg,LAURELS,B-ORG,,
9,000.jpeg,OVERSEAS,I-ORG,,


# text cleaning
* remove whilte spaces
* remove unwanted special character 


In [7]:
whitespace = string.whitespace
punctuation = "!#$%&\'()*+:;<=>?[\\]^`{|}~"
tableWhitespace = str.maketrans('','',whitespace)
tablePunctuation = str.maketrans('','',punctuation)
def cleanText(txt):
    text = str(txt)
    #text = text.lower()
    removewhitespace = text.translate(tableWhitespace)
    removepunctuation = removewhitespace.translate(tablePunctuation)
    
    return str(removepunctuation)

In [8]:
df['text'] = df['text'].apply(cleanText)

In [9]:
dataClean = df.query("text != '' ")
dataClean.dropna(inplace=True)

C:\Users\Shubham\AppData\Local\Temp\ipykernel_5260\3379659824.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataClean.dropna(inplace=True)


In [10]:
dataClean.head(100)

,id,text,tag,,
1,000.jpeg,.,O,,
2,000.jpeg,040-4852,B-PHONE,,
3,000.jpeg,"""8881,""",I-PHONE,,
4,000.jpeg,90309,B-PHONE,,
5,000.jpeg,52549,I-PHONE,,
...,...,...,...,...,...
129,008.jpeg,Telangana,I-ADD,,
130,008.jpeg,-,I-ADD,,
131,008.jpeg,505172,I-ADD,,
132,008.jpeg,email,O,,


# convert data into spacy format

In [11]:
group = dataClean.groupby(by='id')

In [12]:
cards = group.groups.keys()

In [13]:
allCardsData = []
for card in cards:
    cardData = []
    grouparray = group.get_group(card)[['text','tag']].values
    content = ''
    annotations = {'entities':[]}
    start = 0
    end = 0
    for text, label in grouparray:
        text = str(text)
        stringLength = len(text) + 1

        start = end
        end = start + stringLength

        if label != 'O':
            annot = (start,end-1,label)
            annotations['entities'].append(annot)

        content = content + text + ' '
        
        
    cardData = (content,annotations)
    allCardsData.append(cardData)

In [14]:
allCardsData

[('. 040-4852 "8881," 90309 52549 Fi /laurelsoverseaseducation LAURELS OVERSEAS EDUCATIONAL CONSULTANCY PVT. LTD. Sea U.K AUSTRALIA CANADA IRELAND www.laurelseducation.com info@laurelseducation.com ',
  {'entities': [(2, 10, 'B-PHONE'),
    (11, 18, 'I-PHONE'),
    (19, 24, 'B-PHONE'),
    (25, 30, 'I-PHONE'),
    (60, 67, 'B-ORG'),
    (68, 76, 'I-ORG'),
    (77, 88, 'I-ORG'),
    (89, 100, 'I-ORG'),
    (101, 105, 'I-ORG'),
    (106, 110, 'I-ORG'),
    (144, 168, 'B-WEB'),
    (169, 194, 'B-EMAIL')]}),
 ('john smith marketing manager web www.psdgraphics.com phone 123-456-7890 mail email@psdgraphics.com ',
  {'entities': [(0, 4, 'B-NAME'),
    (5, 10, 'I-NAME'),
    (11, 20, 'B-DES'),
    (21, 28, 'I-DES'),
    (33, 52, 'B-WEB'),
    (59, 71, 'B-PHONE'),
    (77, 98, 'B-EMAIL')]}),
 ('Sau 0 98489 24441 dy "08672," 224441 /ENKATESWAPA wie ',
  {'entities': [(6, 11, 'B-PHONE'), (12, 17, 'I-PHONE'), (37, 49, 'B-ORG')]}),
 ('Prasad @ "9,96,31,73,53,59,49,04,00,000" i Flex Design Album Des

In [15]:
card_data_df = pd.DataFrame(allCardsData,columns=['text','labels'])
card_data_df['isNull'] = card_data_df['labels'].apply(lambda x: 'Null' if len(x['entities']) ==0 
                                                      else 'Clean')

# NUll entiries need to drop

In [16]:
card_data_df.query('isNull == "Null"')

,text,labels,isNull
27,ae Be 3,{'entities': []},Null


In [17]:
card_data_df.dropna(inplace=True)
clean_data = card_data_df.query('isNull == "Clean"')[['text','labels']]

In [18]:
allCardsData = list(map(lambda x: tuple(x), clean_data.values.tolist()))

# test and train set

In [19]:
import random

In [20]:
random.shuffle(allCardsData)

In [21]:
len(allCardsData)

266

In [22]:
TrainData = allCardsData[:240]
TestData = allCardsData[240:]

In [23]:
import pickle

In [24]:
pickle.dump(TrainData,open('./data/TrainData.pickle',mode='wb'))
pickle.dump(TestData,open('./data/TestData.pickle',mode='wb'))